This project analyze 911 calls from Montgomery County, PA, published on [Kaggle](https://www.kaggle.com/mchirico/montcoalert). 

Data are in simple CSV file.
Columns description:
* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)

In [2]:
!git clone https://github.com/Burdin84/datasets

Cloning into 'datasets'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
!ls

datasets  sample_data


In [4]:
cd datasets

/content/datasets


In [5]:
!ls
!pwd

 911.csv	    College_Data	       loan_data.csv
 advertising.csv   'Ecommerce Customers'       titanic.csv
'Classified Data'  'Ecommerce Customers.csv'   winequality-red.csv
/content/datasets


Import of libraries

In [7]:
#Data manipulation libraries
import numpy as np
import pandas as pd

#Visualisation libraries
import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv('911.csv')

EDA

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99492 entries, 0 to 99491
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lat        99492 non-null  float64
 1   lng        99492 non-null  float64
 2   desc       99492 non-null  object 
 3   zip        86637 non-null  float64
 4   title      99492 non-null  object 
 5   timeStamp  99492 non-null  object 
 6   twp        99449 non-null  object 
 7   addr       98973 non-null  object 
 8   e          99492 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 6.8+ MB


In [10]:
df.head()

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1


Let's pose a questions. \
What are top 5 zipcodes for 911 calls? \
What are the top 5 townships(twp) for 911 calls?

In [ ]:
df["zip"].value_counts().head(5)

In [ ]:
df["twp"].value_counts().head(5)

How many unique codes we have in column "title"

In [ ]:
df["title"].nunique()

110, it would be good to separate reasons why was 911 called, as an EMS, Fire etc.

Feature ingeneering

In [ ]:
df[['reason','title']]=df['title'].str.split(':',expand=True).apply(lambda x: x.str.strip())

In [ ]:
df.head()

So, What is the most common reason for a 911 call?

In [ ]:
height = df.reason.value_counts()
height

In [ ]:
sns.countplot(x = "reason", data = df)

It would be good to also know year, month, day and hour, so let's go a check timeStamp column

In [ ]:
df["timeStamp"][1]

In [ ]:
type(df["timeStamp"][1])

It's a string object, so we need to change it to DateTime object, let's try to use pd.to_datetime for conversion

In [ ]:
df["timeStamp"] = pd.to_datetime(df["timeStamp"], yearfirst=True, format = "%Y/%m/%d")

In [ ]:
df.head()

In [ ]:
time = df["timeStamp"].iloc[0]

In [ ]:
time.day

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)

In [ ]:
df["Day"] = df["timeStamp"].apply(lambda time: time.dayofweek)
df["Month"] = df["timeStamp"].apply(lambda time: time.month)
df["Year"] = df["timeStamp"].apply(lambda time: time.year)

In [ ]:
df.head()

Afte conversion Day is in integer values, for better comprehension it would be nice to change it to names of days

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df["Day"] = df["Day"].map(dmap)

In [ ]:
df.head()

In [ ]:
sns.countplot(x = "Day", hue = "reason", data = df, palette = "viridis")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.countplot(x = "Month", hue = "reason", data = df, palette = "viridis")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

Hey, where is September, October and November? \
Let's presume that trend will be similar as on previous graph and use count() method for aggregation

In [ ]:
byMonth = df.groupby("Month").count()

In [ ]:
byMonth["twp"].plot()

In [ ]:
sns.lmplot(x = 'Month', y="twp", data = byMonth.reset_index())

In [ ]:
df['Date']=df['timeStamp'].apply(lambda t: t.date())
df.head()

In [ ]:
byDate = df.groupby("Date").count()
byDate.head()

In [ ]:
byDate['twp'].plot()
plt.tight_layout

In [ ]:
df[df["reason"] == "Traffic"].groupby("Date").count()["twp"].plot()
plt.title('Traffic')
plt.tight_layout()

In [ ]:
df[df["reason"] == "Fire"].groupby("Date").count()["twp"].plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
df[df["reason"] == "EMS"].groupby("Date").count()["twp"].plot()
plt.title('EMS')
plt.tight_layout()

It would be nice to see this data in heatmap, but for this we must restructure the data first.\
Columns will be hours and index will be day of the week.

In [ ]:
dayHour = df.groupby(by=['Day','Hour']).count()['reason'].unstack()
dayHour.head()

In [ ]:
sns.heatmap(dayHour, cmap = "viridis")

In [ ]:
sns.clustermap(dayHour, cmap = "viridis")

In [ ]:
dayMonth = df.groupby(by=['Day','Month']).count()['reason'].unstack()
dayMonth.head()

In [ ]:
sns.heatmap(dayMonth, cmap = "viridis")

In [ ]:
sns.clustermap(dayMonth, cmap="viridis")

It looks that worst situation is from Monday to Friday from 3pm to 6pm, I would say this will be caused mostly by traffic and end of working shift, especially in offices. \
From monthly point of view it looks like worst part of the year are Saturdays in January, well probably New Year depresion. 